In [ ]:
import pandas as pd
import pyodbc as odbc
from sqlalchemy import create_engine, MetaData, URL
import matplotlib.pyplot as plt
import seaborn as sns
import pygwalker as pyg

In [ ]:
#Set up path or connection strings here...
#xlsx
path_to_xlsx : str = "./excel_file.xlsx"
path_to_csv : str = "./csv_file.csv"
path_to_json : str = "./json_file.json"

driver : str = "{SQL Server Native Client 11.0}" #"postgresql"
server : str = "127.0.0.1"
pwd : str = "password...should come from an environment variable"
uid : str = "userid... should also come from an environment variable"
database : str = "databasename"

#non ms sql
#connection_url = f'{driver}://{uid}:{pwd}@{server}/{database}'
    
#mssql
connection_string = 'DRIVER=' + driver + ';SERVER=' + server + ';DATABASE=' + database + ';UID=' + uid + ';PWD=' + pwd
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

###Load Data

In [ ]:
#data = pd.read_excel(path_to_xlsx)
#data = pd.read_csv(path_to_csv)
#data = pd.read_json(path_to_json)
sql : str = "Select * from sys.tables"
data = pd.read_sql(sql, conn)

###Data Cleaning

In [ ]:
#drop duplicate records
data = data.drop_duplicates()


In [ ]:
#check out what you have
data.head(10)

In [ ]:
#remove anything you don't need
data = data.drop(columns = "not needed data column")

In [ ]:
#split out any merged data points 
data[["New Col 1", "New Col2", "New Col3"]] = data["source col"].str.split(',',2, expand=True)

In [ ]:
#figure out what to do with nulls
#defaulting to empty string but what if it should be numeric? you know use common sense
data=data.fillna('')


In [ ]:
#do any clean up on data
data['col']  = data['col'].str.replace('search for', 'replace with')

In [ ]:
#does any data now need to be dropped for any reason
for x in data.index:
    if data.loc[x, "column to look at"] == 'reason for dropping it': #great place for a match
        data.drop(x, inplace=True) #drop that record

In [ ]:
#do we have the unique index???
#Yes an id column exists
data.set_index('column name for index', inplace=True)
#composite key?
#data.set_index(['col1', 'col2'], inplace=True)
#nope but we did screw with things so lets reset.
#data.reset_index(inplace=True)

In [ ]:
#just sort it to make searches faster or scrolling through it easier.
data.sort_index(inplace=True)

###Now we can get to looking at some stuff

In [ ]:
#what are we looking at
data.info()

In [ ]:
#how much is left?
data.shape

In [ ]:
#over view of data
data.describe()

In [ ]:
#do we have any missing data
data.isnull().sum()

In [ ]:
#how unique are the columns?
data.nunique()

In [ ]:
#what are the columns?
data.columns

In [ ]:
#get the first 10 records
data.head(10)

In [ ]:
#get the last 10 records
data.tail(10)

In [ ]:
#lets look at a heatmap see what might correlate to what else.
sns.heatmap(data.corr(), annot = True)

plt.rcParams['figure.figsize'] = (20,7)

plt.show()

In [ ]:
#that takes a long time to one by one go through things... 
#just load up pyg and play with the data.
pyg.walk(data, dark='dark')